In [16]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from typing import List
from typing import Optional
from sqlalchemy import URL
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
import psycopg2



In [17]:
db_url = URL.create(
    drivername='postgresql+psycopg2', 
    username = 'postgres', 
    password='CbassEngineering1003.', 
    host='localhost', 
    port="5432", 
    database='steam_games_details'
)

engine = create_engine(db_url)

Base = DeclarativeBase

class Base(DeclarativeBase): 
    pass

class steamReviews(Base): 
    __tablename__ = 'reviews'
    id = mapped_column(Integer, primary_key=True)
    appid = mapped_column(Integer)
    forever_playtime = mapped_column(Integer)
    past_two_week_playtime = mapped_column(Integer)
    review = mapped_column(String)
    
Base.metadata.create_all(engine)
Session = sessionmaker(engine)
session = Session()

try:
    connection = engine.connect()
    print("Connection successful!")
    connection.close()
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


In [18]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import requests
import time
import pandas as pd
import numpy as np
import json
import sqlite3
import pickleshare


In [19]:
%store -r app_ids
print(app_ids)

['730', '570', '578080', '2246340', '252490', '3241660', '2767030', '359550', '271590', '1172470', '236390', '2669320', '3240220', '431960', '221100', '2252570', '1086940', '2943650', '413150', '1771300', '1366800', '440', '1938090', '230410', '394360', '2507950', '227300', '1245620', '2923300', '1222670', '381210', '1174180', '1782210', '2429640', '1085660', '2001120', '438100', '289070', '1091500', '629520', '2694490', '553850', '105600', '892970', '1623730', '1281930', '2300320', '346110', '489830', '108600', '1203220', '251570', '3419430', '3205720', '250900', '2399830', '252950', '4000', '294100', '1142710', '1158310', '2379780', '2357570', '322330', '526870', '238960', '284160', '550', '322170', '739630', '2139460', '427520', '1326470', '582660', '813780', '393380', '582010', '379430', '1295660', '2878980', '513710', '236850', '261550', '1238840', '39210', '292030', '2074920', '1449850', '8930', '240', '10', '1905180', '377160', '990080', '242760', '306130', '1623660', '1551360',

In [20]:
user_agent = "Microsoft Edge UA string:Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.10136"
options = webdriver.EdgeOptions()
options.add_argument('user-agent={0}'.format(user_agent))
options.add_argument("--headless=new")

driver = webdriver.Edge()

url = "https://store.steampowered.com/charts/mostplayed/"

links = []
app_ids = []

def get_app_ids(url, rec=False):
    page = driver.get(url)
    response = requests.get(url, user_agent)
    print(response.status_code, url)


    time.sleep(2)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN) 

        
    soup = BeautifulSoup(driver.page_source, "html")

    chart = soup.find("div", class_ = "_1A7NagdRz58_o8HPHMa3eE").find("table", class_ = "_3arZn0BMPzyhcYNADe193m")

    for ls in chart.find_all("td"):
        try:  
            link = (ls.find("a", href = True)["href"])
            links.append(link)
            app_id = link.split('/')[4]
            app_ids.append(app_id)

        except Exception as e:   
            None

get_app_ids(url)
    
print(app_ids)


driver.quit()

200 https://store.steampowered.com/charts/mostplayed/
['730', '570', '578080', '2246340', '252490', '3241660', '2767030', '359550', '271590', '1172470', '236390', '2669320', '3240220', '431960', '221100', '2252570', '1086940', '2943650', '1771300', '413150', '1366800', '440', '1938090', '230410', '394360', '2507950', '227300', '1245620', '2923300', '1222670', '381210', '1174180', '1782210', '2429640', '1085660', '2001120', '438100', '289070', '1091500', '629520', '2694490', '553850', '892970', '105600', '1623730', '1281930', '2300320', '346110', '489830', '108600', '251570', '3419430', '1203220', '3205720', '250900', '252950', '2399830', '4000', '1158310', '1142710', '294100', '2379780', '2357570', '322330', '526870', '284160', '238960', '550', '739630', '322170', '2139460', '427520', '1326470', '582660', '813780', '393380', '379430', '1295660', '582010', '513710', '2878980', '236850', '39210', '1238840', '261550', '292030', '1449850', '240', '2074920', '8930', '10', '377160', '1905180

In [21]:
def get_reviews(app_id):
    url = f"https://store.steampowered.com/appreviews/{app_id}?json=1"
    response = requests.get(url)
    data = response.json()
    
    Users = data['reviews']

    return Users

In [22]:
for app_id in app_ids: 
    for Users in get_reviews(app_id): 
        print(Users)

{'recommendationid': '189813978', 'author': {'steamid': '76561199483863905', 'num_games_owned': 0, 'num_reviews': 29, 'playtime_forever': 50240, 'playtime_last_two_weeks': 14194, 'playtime_at_review': 40799, 'last_played': 1742229004}, 'language': 'english', 'review': "Counter-Strike 2 is the long-awaited upgrade to one of the most iconic FPS games in history. Built on the Source 2 engine, it brings improved graphics, enhanced responsiveness, and a reworked smoke grenade system that adds more tactical depth. But does it truly feel like a step forward?\n\nPros:\n✅ Visual & Performance Upgrade: The move to Source 2 provides crisper visuals, better lighting, and improved physics, making the game feel more modern while retaining its classic style.\n✅ Responsive Gameplay: CS2 maintains the tight, skill-based mechanics that make Counter-Strike a masterpiece in competitive shooters.\n✅ New Smoke Mechanics: Dynamic smoke grenades that interact with bullets and explosions add fresh strategic po

In [38]:
for app_id in app_ids: 
    for Users in get_reviews(app_id): 
        steam_reviews = steamReviews(
        appid = app_id,
        forever_playtime = Users['author']['playtime_forever'],
        past_two_week_playtime = Users['author']['playtime_last_two_weeks'],
        review = Users['review'])
        session.add(steam_reviews)

        

try: 
    session.commit()
except: 
    session.rollback()

       

In [25]:

query = session.query(steamReviews)

print(query)

SELECT reviews.id AS reviews_id, reviews.appid AS reviews_appid, reviews.forever_playtime AS reviews_forever_playtime, reviews.past_two_week_playtime AS reviews_past_two_week_playtime, reviews.review AS reviews_review 
FROM reviews


In [26]:
from sqlalchemy import select

stmt = select(steamReviews).where(steamReviews.appid == "570")
result = session.execute(stmt)

for steamReviews_obj in result.scalars(): 
    print(f"{steamReviews_obj.appid} {steamReviews_obj.forever_playtime} {steamReviews_obj.review}")

570 226927 They try to make it more stupid and more stupid, now rivers, after that ocean.
570 70388 A perfect game to "come back to". When every other game gets boring. You can allways come back for a round of Dota 2.
570 159302 This is the game that i will probably play till my time on this earth expires
570 8012 new map bad, give me option to play old map
570 2602 toxic, auto losses because trolls exist
570 46940 unbalanced

Play this game only if you hate yourself otherwise play League of Legends.
570 171183 good
570 441105 love it

570 598450 FREE GAME
570 41205 помойка
570 412281 Shit
